# Library imports, configurations

In [37]:
import os
import re
import s3fs
import pandas as pd
import requests
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
from unidecode import unidecode
from wordcloud import WordCloud
import nltk
nltk.download('punkt')

from stop_words import get_stop_words
import unicodedata

from nltk.tokenize import word_tokenize
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch_dsl import connections
from bs4 import BeautifulSoup as bs

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [4]:
BUCKET = "jplaton/diffusion"
fs.ls(BUCKET)

['jplaton/diffusion/.keep',
 'jplaton/diffusion/ted',
 'jplaton/diffusion/visio_mel']

In [5]:
BUCKET_CLEM = "clementg/diffusion"
fs.ls(BUCKET_CLEM)

['clementg/diffusion/.keep',
 'clementg/diffusion/Fiche de Poste DINUM - francetransfert-3595970061- pdf.csv']

# Functions

In [6]:
# Suppression des accents

def remove_accent(s):

    s = unidecode(str(s))
    return(s)

# Suppression de la ponctuation

def remove_punctuation(s):
    # On retire la ponctuation
 
    s = re.sub(r'[^A-Za-z0-9]',' ',str(s))
    return(s)

# Réduction de la casse

def tolower(s):
    s = ' '.join(str(s).lower().split())
    return(s)


# Sur cette base, on crée une petite fonction qui retire les stop words
stopwords = get_stop_words('french')   
stopwords = [unicodedata.normalize('NFKD',m).encode('ASCII', 'ignore').decode() for m in stopwords]

def remove_stopwords(s):
    s = [w for w in word_tokenize(s) if w not in stopwords]
    s = ' '.join(s)
    return(s)

def remove_accent_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(remove_accent)
    return df_c

def remove_punctuation_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(remove_punctuation)
    return df_c

def tolower_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(tolower)
    return df_c

# DATA Download

In [7]:
FILE_PATH_TED = "ted"
FILE_PATH_TED_S3 = BUCKET + "/" + FILE_PATH_TED

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20230409.csv", mode="rb") as file_in:
    offres_2023_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20221225.csv", mode="rb") as file_in:   
    offres_2022_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/Bulloterie_10_Saison_2.xlsx", mode="rb") as file_in:  
    bulloterie_df =  pd.read_excel(file_in, sheet_name=0,header=[0,1])


/tmp/ipykernel_10010/4280061390.py:5: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2023_df = pd.read_csv(file_in, sep=";")
/tmp/ipykernel_10010/4280061390.py:8: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2022_df = pd.read_csv(file_in, sep=";")


In [8]:
with fs.open(BUCKET_CLEM+"/Fiche de Poste DINUM - francetransfert-3595970061- pdf.csv", mode="rb") as file_in:
    fdp_dinum_df = pd.read_csv(file_in, sep=";",encoding='Windows-1252')

In [9]:
with fs.open(FILE_PATH_TED_S3+"/dila_refOrga_admin_Etat_fr_20230505.json", mode="rb") as file_in:  
    orga =  json.load(file_in)

### Traitement référentiel services

In [10]:
orga_df = pd.json_normalize(orga,record_path='service')

In [23]:
orga_df.head(10)

,plage_ouverture,site_internet,copyright,siren,ancien_code_pivot,texte_reference,partenaire,telecopie,nom,siret,itm_identifiant,sigle,date_modification,adresse_courriel,service_disponible,organigramme,pivot,partenaire_identifiant,ancien_identifiant,id,ancien_nom,commentaire_plage_ouverture,annuaire,hierarchie,categorie,sve,telephone_accessible,version_type,type_repertoire,telephone,version_etat_modification,date_creation,partenaire_date_modification,mission,formulaire_contact,version_source,type_organisme,code_insee_commune,statut_de_diffusion,adresse,information_complementaire,date_diffusion
0,[],"[{'libelle': '', 'valeur': 'http://www.police-nationale.interieur.gouv.fr'}]",Direction de l'information légale et administrative (Première ministre),,,[],,[01 47 44 97 99],Bureau de coordination stratégique,,184318,,04/02/2021 18:26:23,[],,[],[],,[],00014eb0-f7fe-4988-9380-137ccdfc401e,[],,[],[],SI,[https://contacts-demarches.interieur.gouv.fr],[],Publiable,,"[{'valeur': '01 49 27 49 27', 'description': ''}]",,10/08/2009 00:00:00,,,[http://www.police-nationale.interieur.gouv.fr/Menu-accessibilite/Nous-contacter],,Administration centrale (ou Ministère),,true,"[{'type_adresse': 'Adresse', 'complement1': '', 'complement2': '', 'numero_voie': '11 rue des Saussaies', 'service_distribution': '', 'code_postal': '75008', 'nom_commune': 'Paris', 'pays': '', 'continent': '', 'longitude': '', 'latitude': '', 'accessibilite': '', 'note_accessibilite': ''}]",,
1,[],"[{'libelle': '', 'valeur': 'https://www.dila.premier-ministre.gouv.fr'}]",Direction de l'information légale et administrative (Première ministre),130009186,,"[{'libelle': 'Missions de la DILA (décret)', 'valeur': 'http://www.legifrance.gouv.fr/affichTexte.do?cidTexte=JORFTEXT000021658499'}, {'libelle': 'Organisation de la DILA (arrêté)', 'valeur': 'https://www.legifrance.gouv.fr/affichTexte.do;?cidTexte=JORFTEXT000033781192'}]",,[],Direction de l'information légale et administrative,13000918600011,185945,DILA,17/03/2023 14:31:25,[],,[],[],,[],0001945a-3c57-4271-b3cd-38ce8ff2c696,[],,[],"[{'type_hierarchie': 'Service Fils', 'service': '68c54a12-2377-4d44-b170-c210312d37b6'}, {'type_hierarchie': 'Service Fils', 'service': '53735d1c-3257-47ad-9197-63fd7edfac36'}, {'type_hierarchie': 'Service Fils', 'service': 'd77756bc-6c8d-4a4f-8a1d-5fefe11a6729'}, {'type_hierarchie': 'Service Fils', 'service': '8df3ee3a-3db4-4bda-9780-89d958c74321'}]",SI,[https://www.dila.premier-ministre.gouv.fr/contacts/saisine-sve],"[{'valeur': 'https://www.acce-o.fr/client/dila', 'description': 'Du lundi au vendredi de 9h à 17h30', 'custom_dico1': ['Transcription automatique']}]",Publiable,,"[{'valeur': '01 40 58 75 00', 'description': ''}]",,06/01/2010 00:00:00,,"La direction de l’information légale et administrative (DILA) est une direction d’administration centrale des services du Premier ministre, placée sous l’autorité du Secrétaire général du Gouvernement. Créée par décret le 11 janvier 2010, elle résulte de la fusion des directions de La Documentation française et des Journaux officiels. La DILA exerce les missions de diffusion légale, d’édition publique et d’information administrative. Elle publie le Journal officiel de la République française (JORF), édite et diffuse les textes législatifs et réglementaires français, mettant à la disposition de tous, la norme juridique française. Elle garantit également la transparence économique et financière par la publication au niveau national de l’ensemble des informations légales, économiques et financières relatives à la vie des entreprises et au milieu associatif. Elle gère des services d’information qui permettent aux citoyens de connaître leurs droits et obligations et facilitent leurs démarches, via le site web service-public.fr. Elle édite des conventions collectives, des codes, des publications de référence, imprimées ou numériques, favorisant l’accès des citoyens à la vie publique et au débat public.",[https://www.dila.premier-ministre.gouv.fr/contacts/nous

In [25]:
orga_df['services_fils'] = orga_df['hierarchie'].map(lambda x: [x[i]['service'] for i in range(len(x))])

In [28]:
services_clean = ( orga_df
    .pipe(remove_accent_from_df,['nom','type_organisme','sigle'])
    .pipe(remove_punctuation_from_df,['nom','type_organisme','sigle'])
    .pipe(tolower_df,['nom','type_organisme','sigle'])
)

/tmp/ipykernel_10010/700602202.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(remove_accent)
/tmp/ipykernel_10010/700602202.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(remove_punctuation)
/tmp/ipykernel_10010/700602202.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(tolower)


In [29]:
services_clean = services_clean[['nom','type_organisme','sigle','services_fils']]

In [36]:
services_clean.head(5) #len(services_clean[services_clean['services_fils'].map(len)==0])

,nom,type_organisme,sigle,services_fils
0,bureau de coordination strategique,administration centrale ou ministere,,[]
1,direction de l information legale et administrative,administration centrale ou ministere,dila,"[68c54a12-2377-4d44-b170-c210312d37b6, 53735d1c-3257-47ad-9197-63fd7edfac36, d77756bc-6c8d-4a4f-8a1d-5fefe11a6729, 8df3ee3a-3db4-4bda-9780-89d958c74321]"
2,bureau de la simplification et de la qualite de service,administration centrale ou ministere,,[]
3,service de la protection sdlp,administration centrale ou ministere,,[]
4,service national d accueil telephonique de l enfance en danger snated,groupement d interet public,,[]


In [38]:
url="https://lannuaire.service-public.fr/gouvernement/53735d1c-3257-47ad-9197-63fd7edfac36"
response = requests.get(url)

In [40]:
html = response.content
print(html)

b'<!DOCTYPE html><html xmlns:ff4j="http://www.w3.org/1999/xhtml"\n      xmlns:esi="http://www.edge-delivery.org/esi/1.0"\n      class="no-js"\n      dir="ltr" ZZdata-fr-theme="dark"\n      data-fr-js="false" lang="fr"><head>\n\t<title>Sous-direction des syst\xc3\xa8mes d&#39;information - Annuaire | Service-public.fr</title>\n\t<meta charset="utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><meta name="description"\n        content="Sous-direction des syst\xc3\xa8mes d&#39;information : coordonn\xc3\xa9es, site internet, principaux services, nom de leurs responsables"/><meta name="audience" content="annuaire"/><script id="abtasty" type="opt-in" data-type="text/javascript" data-name="ab-tasty"\n            data-src="https://try.abtasty.com/f2a27ecc441fb1662a47bee9ea97d2ea.js"></script><script src="//cdn1.readspeaker.com/script/12082/webReader/webReader.js?pids=wr&notools=1" type="text/javasc

In [44]:
soup = bs(html, "lxml")
print(soup)

<!DOCTYPE html>
<html class="no-js" data-fr-js="false" dir="ltr" lang="fr" xmlns:esi="http://www.edge-delivery.org/esi/1.0" xmlns:ff4j="http://www.w3.org/1999/xhtml" zzdata-fr-theme="dark"><head>
<title>Sous-direction des systèmes d'information - Annuaire | Service-public.fr</title>
<meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><meta content="Sous-direction des systèmes d'information : coordonnées, site internet, principaux services, nom de leurs responsables" name="description"/><meta content="annuaire" name="audience"/><script data-name="ab-tasty" data-src="https://try.abtasty.com/f2a27ecc441fb1662a47bee9ea97d2ea.js" data-type="text/javascript" id="abtasty" type="opt-in"></script><script id="readspeaker_script" src="//cdn1.readspeaker.com/script/12082/webReader/webReader.js?pids=wr&amp;notools=1" type="text/javascript"></script><link href="https://www.service-public.f

In [43]:
soup.title.get_text()

"Sous-direction des systèmes d'information - Annuaire | Service-public.fr"

### Traitement offres

In [46]:
offres_df = pd.concat([offres_2023_df,offres_2022_df])

In [47]:
offres_df.head()

,Organisme de rattachement,Référence,Versant,Métier,Statut du poste,Nature de l'emploi,Durée du contrat,Intitulé du poste,Localisation du poste,Lieu d'affectation,Niveau d'études,Niveau d'expérience min. requis,Date de vacance de l'emploi,Date de début de publication par défaut,Date de fin de publication par défaut,Date de première publication,Spécialisation,Télétravail,Management,Employeur,Catégorie,Temps Plein,Durée du poste,Langues,Niveaux,Lieu d'affectation (sans géolocalisation),Avis de vacances au JO,Compétences attendues,Documents à transmettre,Nature de contrat
0,Structures de coopération territoriale,O043221200865282,Fonction Publique Territoriale,Chargée / Chargé de politique environnementale,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,Géomaticien.ne écologue - SYNDICAT MIXTE DE GESTION DU CONSERVATOIRE BOTANIQUE,Haute Loire (43),1 rue Adrienne de Noailles - 43230 Chavaniac-Lafayette,NaN,NaN,01/03/2023,28/12/2022,31/01/2023,01/12/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Oui,NaN,NaN,NaN,Chavaniac-Lafayette,NaN,NaN,NaN,NaN
1,Structures de coopération territoriale,O043221200864991,Fonction Publique Territoriale,Chargée / Chargé de politique environnementale,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,"Chargé.e mission ""flore/habitats naturels"" végétations agro pastorales",Haute Loire (43),1 rue Adrienne de Noailles - 43230 Chavaniac-Lafayette,NaN,NaN,01/03/2023,28/12/2022,31/01/2023,01/12/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Oui,NaN,NaN,NaN,Chavaniac-Lafayette,NaN,NaN,NaN,NaN
2,Structures de coopération territoriale,O043221200865428,Fonction Publique Territoriale,Chargée / Chargé de politique environnementale,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,Chargé.e de missions scientifiques et techniques,Haute Loire (43),1 rue Adrienne de Noailles - 43230 Chavaniac-Lafayette,NaN,NaN,01/03/2023,28/12/2022,31/01/2023,01/12/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Oui,NaN,NaN,NaN,Chavaniac-Lafayette,NaN,NaN,NaN,NaN
3,Structures de coopération territoriale,O064221200866576,Fonction Publique Territoriale,"Plombière, opératrice, chauffagiste / Plombier, opérateur, chauffagiste en froid et ventilation",Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,Agent de maintenance des bâtiments - Communauté de communes lacq-orthez - CCLO,Pyrénées Atlantiques (64),Hôtel de la communauté de communes de lacq-orthez - 64150 Mourenx,NaN,NaN,01/03/2023,28/12/2022,27/01/2023,02/12/2022,NaN,NaN,NaN,NaN,Catégorie C (employé),Oui,NaN,NaN,NaN,Mourenx,NaN,NaN,NaN,NaN
4,Centres communaux d'Action Sociale,O032221000829833,Fonction Publique Territoriale,Éducatrice / Educateur de jeunes enfants,Vacant,Emploi ouvert uniquement aux contractuels,7 mois,Puériculteur.rice / Educateur.rice de jeunes enfants - C.C.A.S de FLEURANCE,Gers (32),Mairie - 32500 FLEURANCE,NaN,NaN,09/01/2023,27/10/2022,26/12/2022,03/11/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Non,NaN,NaN,NaN,FLEURANCE,NaN,NaN,NaN,NaN


In [15]:
offres_df.columns = offres_df.columns.map(remove_accent).map(remove_punctuation).map(tolower).map(lambda s: s.replace(' ', '_'))

 offres_df[offres_df['metier'].str.contains('scientist|engineer',na=False)].head(2)

In [17]:
offres_df_clean = ( offres_df
    .pipe(remove_accent_from_df,['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste'])
    .pipe(remove_punctuation_from_df,['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste'])
    .pipe(tolower_df,['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste'])
)

In [90]:
offres_df_clean = offres_df_clean[['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste']]

In [105]:
offres_df_clean[offres_df_clean['metier'].str.contains("ux designer",na=False)|offres_df_clean['intitule_du_poste'].str.contains("data designer",na=False)].head(5)

,organisme_de_rattachement,metier,intitule_du_poste,localisation_du_poste
136654,caisse des depots et consignations,architecte technique,expert conception et architecture de donnees d entreprise data designer h f,gironde
136668,caisse des depots et consignations,architecte technique,expert conception et architecture de donnees d entreprise data designer h f,maine et loire
136672,caisse des depots et consignations,architecte technique,expert conception et architecture de donnees d entreprise data designer h f,paris


### Traitement Fiche de poste DNUM

In [84]:
# Traitement des colonnes
fdp_dinum_df.columns = fdp_dinum_df.columns.map(remove_accent).map(remove_punctuation).map(tolower).map(lambda s: s.replace(' ', '_'))

In [86]:
fdp_clean = ( fdp_dinum_df
    .pipe(remove_accent_from_df,['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences'])
    .pipe(remove_punctuation_from_df,['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences'])
    .pipe(tolower_df,['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences'])
)

In [88]:
fdp_clean = fdp_clean[['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences']]
fdp_clean.head(2)


,poste,intitule_du_service_demandeur_bureau_section,corps_grade,presentation,missions,competences
0,gestionnaire financier,direction interministerielle du numerique secretariat general pole budget achats,secretaire administratif ve b,service de la premiere ministre place sous l autorite de la ministre de la transformation et de la fonction publiques la direction interministerielle du numerique dinum est en charge de la transformation numerique de l etat elle accompagne les ministeres conseille le gouvernement et developpe des services et ressources numeriques partagees pour les citoyens et les agents publics rejoindre la dinum c est prendre part a un collectif de pres de 200 agents reunis autour d une meme ambition creer grace au numerique le service public de demain le secretariat general de la dinum est responsable des fonctions transverses de la dinum ressources humaines communication finances logistique juridique garantissant le bon fonctionnement des services de la dinum et la qualite des conditions de travail des agents le pole budget et achats est plus particulierement en charge d assurer l execution financiere des actes de gestion de mettre en oeuvre l achat public et de contribuer a la performance et a la qualite de l execution des operations budgetaires et financieres l ensemble de ces actions porte aujourd hui sur trois programmes budgetaires et bop distincts 129 352 et 349,missions vous assurez l instruction et l execution financiere des actes de gestion de la dinum activites principales instruire saisir et suivre les actes de gestion demandes d achat services faits etc des differents programmes geres par la dinum au sein du logiciel chorus formulaires produire des restitutions et ou alertes financieres zdep56 zbud01 etc assurer l interface entre les differents acteurs de la chaine financiere services comptables et budgetaires des spm et fournisseurs afin d en garantir la fluidite contribuer a la gestion et au suivi des investissements fiec fies operations d inventaire notamment de la dinum suivre en continu le paiement et la cloture des engagements traitement des restes a payer en fonction de l evolution des priorites fixees a la dinum reorganisation en cours il elle peut etre amene e a prendre en charge d autres taches et projets,savoirs maitrise des regles et processus financiers lolf gbcp commande publique une premiere experience en gestion financiere dans la fonction publique est requise savoir faire solides connaissances de chorus coeur et formulaires connaissance et pratique souhaitees en matiere d investissement fiec fies gestion d inventaire gerer et exploiter les bases de donnees savoir etre autonomie rigueur et sens des responsabilites appetence pour le domaine du numerique solides capacites d organisation et de travail en equipe
1,adjoint e du de la chef fe de departement et responsable de l incubation de l operateur de produits interministeriels,direction interministerielle du numerique dinum departement de l operateur de produits interministeriels,,la direction interministerielle du numerique dinum est un service de la premiere ministre place sous l autorite du ministre de la transformation et de la fonction publiques elle oriente anime soutient et coordonne les actions des administrations de l etat et celles des organismes places sous sa tutelle visant a ameliorer la qualite l efficacite l efficience et la fiabilite des services rendus par le systeme d information et de communication de l etat elle contribue a l ouverture des donnees publiques aux strategies d innovation ouverte et au travail avec les ecosystemes d innovation elle pilote la politique de mutualisation du systeme d information et de communication de l etat et peut intervenir dans la gouvernance des operations de mutualisation ainsi que dans la conception l elaboration et la mise en oeuvre de ces operations enfin elle soutient le developpement des competences dans le domaine du numerique en facilitant l emergence d une filiere professio

# Insertion des données dans Elastic Search

In [59]:
# Paramétrage du client
es_client = connections.create_connection(hosts=['http://elasticsearch-master:9200/'])

In [63]:
# Fontion qui transforme un dataframe en document/index à stocker dans ElasticSearch
def doc_generator(df,index_name):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": index_name,
                "_type": "_doc",
                "_source": document,
            }

In [68]:
# Indexation des offres
helpers.bulk(es_client, doc_generator(offres_df_clean,"offres"))

/tmp/ipykernel_52094/3268320283.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es_client, doc_generator(offres_df_clean))
/tmp/ipykernel_52094/3268320283.py:1: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  helpers.bulk(es_client, doc_generator(offres_df_clean))


(406755, [])

In [64]:
# Indexation des services
helpers.bulk(es_client, doc_generator(services_clean,"services"))

/tmp/ipykernel_72898/840881214.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es_client, doc_generator(services_clean,"services"))
/tmp/ipykernel_72898/840881214.py:1: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  helpers.bulk(es_client, doc_generator(services_clean,"services"))


(7636, [])

In [89]:
# Indexation des fiches de poste
helpers.bulk(es_client, doc_generator(fdp_clean,"fiches"))

/tmp/ipykernel_72898/2169517871.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es_client, doc_generator(fdp_clean,"fiches"))
/tmp/ipykernel_72898/2169517871.py:2: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  helpers.bulk(es_client, doc_generator(fdp_clean,"fiches"))


(30, [])

In [130]:
from elasticsearch_dsl import Search
s = Search(index='offres').query('match',metier='charge')

In [131]:
r=s.execute()

/opt/mamba/lib/python3.11/site-packages/elasticsearch_dsl/search.py:712: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.search(index=self._index, body=self.to_dict(), **self._params).body,


In [132]:
df_results = pd.DataFrame((d.to_dict() for d in s.scan()))

/opt/mamba/lib/python3.11/site-packages/elasticsearch_dsl/search.py:728: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  for hit in scan(es, query=self.to_dict(), index=self._index, **self._params):


In [133]:
df_results

,organisme_de_rattachement,metier,intitule_du_poste,localisation_du_poste
0,universite paris saclay,chargee charge de controle interne budgetaire et comptable,gestionnaire des operations financieres h f,essonne
1,structures de cooperation territoriale,chargee charge de developpement territorial,che fe de projet petites villes de demain com com du civraisien en poitou,vienne
2,centres communaux d action sociale,chargee charge de gestion administrative et ou de paie,assistant charge du portage de repas et du logement h f,morbihan
3,conseils departementaux,chargee charge de gestion administrative et ou de paie,secretaire du service crip conseil departemental de la gironde,gironde
4,communes,chargee charge d amenagement des espaces verts et de production vegetale,agent d entretien des espaces verts h f le petit quevilly,seine maritime
...,...,...,...,...
153856,communes,chargee charge de l instruction de dossiers administratifs,instructeur instructrice des demandes autorisations droits des sols commune d aix en provence,bouches du rhone
153857,communes,chargee charge des d exploitation des equipements sportifs et ludiques,chef d equipe exploitation h f lyon,rhone
153858,direction des achats de l etat dae,chargee charge de conduite des politiques publiques,dae consultant interne etudes actions d optimisation des achats de l etat et de ses operateurs h f,paris
153859,communes,chargee charge de l instruction de dossiers administratifs,agent des formalites civiles commune d aix en provence,bouches du rhone


In [96]:
 # Recherche dans l'ensemble des champs le meilleur écho (le plus pertinent) #
fullsearch_easy = es_client.search(index = "services", # l'index dans lequel on cherche
                       q = "direction generale des finances publiques dgfip	data scientis ssi dpn dtnum bureau si mission service a l utilisateur data scientist expert h f", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités

/tmp/ipykernel_72898/3955335858.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch_easy = es_client.search(index = "services", # l'index dans lequel on cherche


In [97]:
fullsearch_easy

ObjectApiResponse({'took': 16, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 7636, 'relation': 'eq'}, 'max_score': 26.99309, 'hits': [{'_index': 'services', '_type': '_doc', '_id': 'bbFaKIsB1My3OOFZFDtC', '_score': 26.99309, '_source': {'copyright': 'direction de l information legale et administrative premiere ministre', 'nom': 'direction generale des finances publiques dgfip', 'type_organisme': 'administration centrale ou ministere', 'sigle': ''}}]}})

In [100]:
 # Recherche dans l'ensemble des champs le meilleur écho (le plus pertinent) #
fullsearch_scientist = es_client.search(index = "services", # l'index dans lequel on cherche
                       q = "centre hospitalier universitaire de tours	data scientist datascientist centre de donnees cliniques", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités

/tmp/ipykernel_72898/2210460514.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch1 = es_client.search(index = "services", # l'index dans lequel on cherche


In [101]:
fullsearch_scientist

ObjectApiResponse({'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 7636, 'relation': 'eq'}, 'max_score': 15.201317, 'hits': [{'_index': 'services', '_type': '_doc', '_id': 'DrFaKIsB1My3OOFZEzmY', '_score': 15.201317, '_source': {'copyright': 'direction de l information legale et administrative premiere ministre', 'nom': 'centre de valorisation des ressources humaines cvrh de tours', 'type_organisme': 'service a competence nationale', 'sigle': ''}}]}})

In [103]:
 # Recherche dans l'ensemble des champs le meilleur écho (le plus pertinent) #
fullsearch_engineer= es_client.search(index = "services", # l'index dans lequel on cherche
                       q = "service hydrographique et oceanographique de la marine	data engineer	ingenieur en traitement de l information conception developpement et experimentation d ia h f", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités

/tmp/ipykernel_72898/1096055904.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch_engineer= es_client.search(index = "services", # l'index dans lequel on cherche


In [104]:
fullsearch_engineer

ObjectApiResponse({'took': 23, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 7636, 'relation': 'eq'}, 'max_score': 27.117067, 'hits': [{'_index': 'services', '_type': '_doc', '_id': 't7FaKIsB1My3OOFZFT--', '_score': 27.117067, '_source': {'copyright': 'direction de l information legale et administrative premiere ministre', 'nom': 'service hydrographique et oceanographique de la marine shom', 'type_organisme': 'etablissement public', 'sigle': ''}}]}})

In [106]:
fullsearch_designer = es_client.search(index = "services", # l'index dans lequel on cherche
                       q = "caisse des depots et consignations	architecte technique expert conception et architecture de donnees d entreprise data designer h f	", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités

/tmp/ipykernel_72898/3294352241.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch_designer = es_client.search(index = "services", # l'index dans lequel on cherche


In [107]:
fullsearch_designer

ObjectApiResponse({'took': 17, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 7636, 'relation': 'eq'}, 'max_score': 21.853615, 'hits': [{'_index': 'services', '_type': '_doc', '_id': 'RbFaKIsB1My3OOFZEDBa', '_score': 21.853615, '_source': {'copyright': 'direction de l information legale et administrative premiere ministre', 'nom': 'caisse des depots et consignations cdc', 'type_organisme': 'juridiction', 'sigle': ''}}]}})

In [118]:
def get_services(es,query):
    fullsearch = es.search(index = "services", # l'index dans lequel on cherche
                       q = query, # notre requête textuelle
                       size = 1)
    return fullsearch['hits']['hits'][0]['_source']['nom']

In [122]:
def add_services(df, list_cols,es):
    df['service'] = df[list_cols].apply(lambda row: get_services(es,' '.join(row.values.astype(str))), axis=1)
    return df

In [119]:
get_services(es_client,"caisse des depots et consignations	architecte technique expert conception et architecture de donnees d entreprise data designer h f")

/tmp/ipykernel_72898/198050217.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch = es.search(index = "services", # l'index dans lequel on cherche


'caisse des depots et consignations cdc'

In [143]:
list_cols = ['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste']
offres_df_temp = offres_df_clean[offres_df_clean['metier'].str.contains("ux designer",na=False)|offres_df_clean['intitule_du_poste'].str.contains("ux designer",na=False)]
offres_with_services =  offres_df_temp.head(50).pipe(add_services,list_cols,es_client)

/tmp/ipykernel_72898/198050217.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch = es.search(index = "services", # l'index dans lequel on cherche
/tmp/ipykernel_72898/2305960098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['service'] = df[list_cols].apply(lambda row: get_services(es,' '.join(row.values.astype(str))), axis=1)


In [ ]:
offres_with_services

In [148]:
list_cols = ['poste','intitule_du_service_demandeur_bureau_section','corps_grade'] #,'presentation','missions','competences'
fdp_temp = fdp_clean[fdp_clean['poste'].str.contains("data engineer",na=False)|fdp_clean['competences'].str.contains("data engineer",na=False)]
fdp_with_services =  fdp_temp.pipe(add_services,list_cols,es_client)

/tmp/ipykernel_72898/198050217.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch = es.search(index = "services", # l'index dans lequel on cherche
/tmp/ipykernel_72898/2305960098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['service'] = df[list_cols].apply(lambda row: get_services(es,' '.join(row.values.astype(str))), axis=1)


In [149]:
fdp_with_services

,poste,intitule_du_service_demandeur_bureau_section,corps_grade,presentation,missions,competences,service
24,ingenieur data f h,direction interministerielle du numerique dinum departement etalab,,la direction interministerielle du numerique dinum est un service de la premiere ministre place sous l autorite du ministre de la transformation et de la fonction publiques elle oriente anime soutient et coordonne les actions des administrations de l etat et celles des organismes places sous sa tutelle visant a ameliorer la qualite l efficacite l efficience et la fiabilite des services rendus par le systeme d information et de communication de l etat elle contribue a l ouverture des donnees publiques aux strategies d innovation ouverte et au travail avec les ecosystemes d innovation elle pilote la politique de mutualisation du systeme d information et de communication de l etat et peut intervenir dans la gouvernance des operations de mutualisation ainsi que dans la conception l elaboration et la mise en oeuvre de ces operations enfin elle soutient le developpement des competences de l etat et des organismes places sous sa tutelle dans le domaine du numerique en facilitant l emergence d une filiere professionnelle interministerielle du numerique rejoindre la dinum c est prendre part a un collectif de plus de 200 agents reunis autour d une meme ambition le numerique au service de l efficacite de l action publique la le titulaire du poste integre le departement etalab de la dinum etalab vise a valoriser et exploiter les donnees comme levier d efficacite de l action publique il coordonne la conception et la mise en oeuvre de la strategie de l etat dans le domaine de la donnee il promeut l action de l etat en matiere d ouverture de circulation et d exploitation des donnees publiques il accompagne les administrations dans l usage de la data science de l intelligence artificielle et diffuse une culture de l innovation ouverte au sein des administrations,au sein du datalab l ingenieur e data travaille avec un product owner des data scientists et un analyste de donnees pour developper des cas d usages mobilisant des donnees et a fort impact au sein d une equipe produit suivant la methode agile l agent a pour mission de mettre en place des outils de traitement de la donnee pour realiser des cas d usage il elle met en oeuvre des traitements sur les donnees manipulees dans le cadre des cas d usage portes par le datalab afin de favoriser et de faciliter la reutilisation de celles ci par son equipe vous serez amene e a utiliser et mettre en oeuvre des outils de traitements et de stockage de donnees bases de donnees pour faciliter leur mise a jour et leur mise a disposition vous serez sensible a la question de la securite et de l administration systeme et gererez cette problematique dans les infrastructures data que vous utiliserez a ce titre vous collaborerez etroitement avec le responsable datasecops f h de l equipe faciliter l interoperabilite des donnees en integrant des variables pivots permettant le croisement avec d autres donnees et vous assurer du respect de bonnes pratiques techniques permettant une meilleure exploitation des donnees pour cela vous integrerez les dimensions de completude d unicite de conformite de tracabilite et de coconformite de tracabilite et de coherence des donnees garantir les bonnes pratiques de developpements appliquees sur les donnees traitements montee en qualite calculs d indicateurs dans l optique de maximiser l impact de projet data a fort potentiel veiller a ce que les outils et traitements developpes dans le cadre du datalab soient transferables aux entites chargees de les executer en production dans les administrations lorsque les cas d usage data ont demontre leur impact et sont finalises mener une veille technologique active pour mobiliser les meilleures technologies au cours de la realisation de vos missions vous devriez etre en veille permanente pour detecter des projets a impact a valoriser ou a accomp